In [1]:
!pip install pycaret

     |████████████████████████████████| 320 kB 8.5 MB/s 
     |████████████████████████████████| 88 kB 7.4 MB/s 
     |████████████████████████████████| 56 kB 3.5 MB/s 
     |████████████████████████████████| 2.0 MB 48.0 MB/s 
     |████████████████████████████████| 16.8 MB 813 kB/s 
     |████████████████████████████████| 261 kB 46.8 MB/s 
     |████████████████████████████████| 6.8 MB 40.9 MB/s 
     |████████████████████████████████| 118 kB 58.8 MB/s 
     |████████████████████████████████| 167 kB 53.1 MB/s 
     |████████████████████████████████| 1.3 MB 49.6 MB/s 
     |████████████████████████████████| 1.7 MB 31.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 690 kB 57.3 MB/s 
     |████████████████████████████████| 636 kB 61.2 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go
import datetime
plt.style.use('fivethirtyeight') 
%matplotlib inline

In [27]:
data = pd.read_csv('/content/train.csv')
data['hour']  = [ str(i)+':0:0' for i in data['hour']]
data['datetime'] =  pd.to_datetime(data['date'] + ' ' + data['hour'])
#data = data[['datetime','demand']]
data.head()

,date,hour,demand,datetime
0,2018-08-18,9:0:0,91,2018-08-18 09:00:00
1,2018-08-18,10:0:0,21,2018-08-18 10:00:00
2,2018-08-18,13:0:0,23,2018-08-18 13:00:00
3,2018-08-18,14:0:0,104,2018-08-18 14:00:00
4,2018-08-18,15:0:0,81,2018-08-18 15:00:00


# created Extra Feature weekday and holiday 

In [28]:
weekday = []
for i in range(data.shape[0]):
  a,b,c = data['date'][i].split("-")
  today = datetime.datetime(int(a),int(b),int(c))
  if (today.isoweekday() == 6) | (today.isoweekday() == 7):
    weekday.append(1)
  else:
    weekday.append(0)

data['weekday'] = weekday

In [6]:
#import holidays
# getting India holidays
#india_holidays = holidays.India()

#holiday = []
#for i in range(data.shape[0]):
  #a = data['date'][i]
  #if (india_holidays.get(a) == None):
    #holiday.append(0)
  #else:
    #holiday.append(1)

#data['holiday'] = holiday

In [31]:
test_data =pd.read_csv("/content/test.csv")
test_data['hour']  = [ str(i)+':0:0' for i in test_data['hour']]
test_data['datetime'] =  pd.to_datetime(test_data['date'] + ' ' + test_data['hour'])
#test_data = pd.DataFrame(test_data['datetime'])
test_data.head()

,date,hour,datetime
0,2021-03-01,0:0:0,2021-03-01 00:00:00
1,2021-03-01,1:0:0,2021-03-01 01:00:00
2,2021-03-01,2:0:0,2021-03-01 02:00:00
3,2021-03-01,3:0:0,2021-03-01 03:00:00
4,2021-03-01,5:0:0,2021-03-01 05:00:00


In [32]:
weekday_test = []
for i in range(test_data.shape[0]):
  a,b,c = test_data['date'][i].split("-")
  today = datetime.datetime(int(a),int(b),int(c))
  if (today.isoweekday() == 6) | (today.isoweekday() == 7):
    weekday_test.append(1)
  else:
    weekday_test.append(0)

test_data['weekday'] = weekday_test

In [9]:
#holiday_test = []
#for i in range(test_data.shape[0]):
  #a = test_data['date'][i]
  #if (india_holidays.get(a) == None):
    #holiday_test.append(0)
  #else:
    #holiday_test.append(1)

#test_data['holiday'] = holiday_test

In [33]:
data.shape,test_data.shape

((18247, 5), (7650, 4))

In [35]:
df = pd.concat([data,test_data],axis=0)
df = df[['datetime','weekday','demand']]
#df = df.set_index("datetime")
df.head()

,datetime,weekday,demand
0,2018-08-18 09:00:00,1,91.0
1,2018-08-18 10:00:00,1,21.0
2,2018-08-18 13:00:00,1,23.0
3,2018-08-18 14:00:00,1,104.0
4,2018-08-18 15:00:00,1,81.0


# Train Test Split Data

In [36]:
train_size = 18247

test_size = len(df) - train_size
train, test = df.iloc[0:train_size], df.iloc[train_size:len(df)]
print(len(train), len(test))

18247 7650


# pycaret Model

In [47]:
from pycaret.regression import *
reg_test = setup(data=train,target='demand',use_gpu=True,fold_strategy = 'timeseries', transform_target = True)

,Description,Value
0,session_id,6064
1,Target,demand
2,Original Data,"(18247, 3)"
3,Missing Values,False
4,Numeric Features,0
5,Categorical Features,1
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(12772, 46)"


In [48]:
best = compare_models(sort = 'MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,29.0152,1336.6285,36.5393,0.2320,0.6289,0.7937,0.123
gbr,Gradient Boosting Regressor,29.3001,1387.2087,37.2194,0.2041,0.6354,0.8203,0.560
br,Bayesian Ridge,29.3003,1383.1589,37.1637,0.2069,0.6327,0.8051,0.074
huber,Huber Regressor,29.3041,1383.8894,37.1729,0.2064,0.6392,0.8314,0.195
ridge,Ridge Regression,29.3045,1380.8072,37.1319,0.2082,0.6322,0.8012,0.060
lar,Least Angle Regression,29.3063,1380.6248,37.1295,0.2083,0.6321,0.8007,0.054
lr,Linear Regression,29.5345,1405.9756,37.4675,0.1923,0.6365,0.8084,0.078
omp,Orthogonal Matching Pursuit,31.3296,1649.5094,40.5851,0.0542,0.6757,0.9090,0.040
rf,Random Forest Regressor,31.4284,1608.7147,40.0945,0.0746,0.6777,0.8406,2.146
knn,K Neighbors Regressor,31.4799,1610.1971,40.1058,0.0753,0.6789,0.8920,0.575


In [49]:
prediction_holdout = predict_model(best);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,29.1346,1299.7411,36.0519,0.2221,0.632,0.8078


In [50]:
pred = predict_model(best,data=test)
pred.head()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,0,0,0,0,0,0


,datetime,weekday,demand,Label
0,2021-03-01 00:00:00,0,NaN,62.653953
1,2021-03-01 01:00:00,0,NaN,55.417491
2,2021-03-01 02:00:00,0,NaN,52.133315
3,2021-03-01 03:00:00,0,NaN,52.456402
4,2021-03-01 05:00:00,0,NaN,53.909475


In [51]:
test_data = pd.read_csv("/content/test.csv")
test_data['demand'] = round(pred['Label'])
#test_data['demand'] = pred['Label']
test_data.head()

,date,hour,demand
0,2021-03-01,0,63.0
1,2021-03-01,1,55.0
2,2021-03-01,2,52.0
3,2021-03-01,3,52.0
4,2021-03-01,5,54.0


In [52]:
test_data.to_csv("solution5.csv",index=False)